# Grant: Per-probe parameter editing + post-run plot/GUI reload

Use this notebook to:
1. edit `grant_recording_config.json` per-probe/per-mode overrides from notebook cells,
2. resolve `ks_dir` and `save_path` for any run mode + probe,
3. reload and rerun:
   - `bc.plot_summary_data(...)` (correct spelling; not `plot_sumary_data`)
   - `bc.compare_manual_vs_bombcell(save_path)`
   - `bc.unit_quality_gui(...)` (main interactive GUI).


In [ ]:
from pathlib import Path
import json
import copy

import bombcell as bc

CONFIG_FILE = Path('../configs/grant_recording_config.json')
RUN_MODE = 'batch'  # batch | single_probe | np20_rerun
TARGET_PROBE = 'A'  # A-F
SAVE_CONFIG = False  # set True only when you want to write config updates


In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str((Path.cwd() / '..' / '..').resolve()))

from grant.grant_config import load_grant_config

cfg = load_grant_config(CONFIG_FILE)
print('Loaded config:', cfg['config_path'])
print('Recording:', cfg['recording_name'])
print('Target probe:', TARGET_PROBE)
print('Run mode:', RUN_MODE)


## 1) Edit per-probe/per-mode overrides + ROI inside notebook

- `probe_param_overrides[probe]['all_modes']` applies to every mode for that probe.
- `probe_param_overrides[probe]['modes'][mode]` applies only for one mode.
- `probe_recording_roi[probe]` stores ROI length from tip (um) in the JSON config.
- Set `SAVE_CONFIG=True` to persist changes back to JSON.


In [ ]:
raw = json.loads(CONFIG_FILE.read_text(encoding='utf-8'))
probe_cfg = raw.setdefault('probe_param_overrides', {}).setdefault(TARGET_PROBE, {})
all_modes = probe_cfg.setdefault('all_modes', {})
modes = probe_cfg.setdefault('modes', {})
this_mode = modes.setdefault(RUN_MODE, {})

# ROI from tip (um) per probe
probe_roi = raw.setdefault('probe_recording_roi', {})

# ---- Example edits (change values as needed) ----
all_modes['minPresenceRatio'] = all_modes.get('minPresenceRatio', 0.7)
this_mode['maxRPVviolations'] = this_mode.get('maxRPVviolations', 0.2)
probe_roi[TARGET_PROBE] = probe_roi.get(TARGET_PROBE, 950)

print('all_modes overrides for probe', TARGET_PROBE, ':', all_modes)
print(f"mode-specific overrides for probe {TARGET_PROBE} / {RUN_MODE}:", this_mode)
print(f"ROI (um from tip) for probe {TARGET_PROBE}:", probe_roi[TARGET_PROBE])

if SAVE_CONFIG:
    CONFIG_FILE.write_text(json.dumps(raw, indent=2), encoding='utf-8')
    print('Saved updated config to', CONFIG_FILE)
else:
    print('SAVE_CONFIG=False -> no file written')


## 2) Resolve run paths for this mode/probe

The unified runner now saves Bombcell outputs directly in:
- `.../kilosort4_{probe}/bombcell/` for `batch`
- `.../kilosort4_{probe}/bombcell/` for `np20_rerun`
- `.../kilosort4_{probe}/bombcell/` for `single_probe`


In [ ]:
mode_to_roots = {
    'batch': cfg['default_ks_staging_root'],
    'np20_rerun': cfg['np20_ks_staging_root'],
    'single_probe': cfg['bombcell_singleprobe_root'],
}

staging_root = mode_to_roots[RUN_MODE]
ks_dir = Path(staging_root) / f'kilosort4_{TARGET_PROBE}'
save_path = ks_dir / 'bombcell'

print('ks_dir:', ks_dir)
print('save_path:', save_path)
print('ks_dir exists:', ks_dir.exists())
print('save_path exists:', save_path.exists())


## 3) Reload everything needed for summary plots + GUI


In [ ]:
# Reload parameter + quality metrics from saved bombcell outputs
param, quality_metrics, _ = bc.load_bc_results(str(save_path))

# Reload template waveforms from staged kilosort directory
_, _, template_waveforms, _, _, _, _ = bc.load_ephys_data(str(ks_dir))

# Recompute unit type labels from loaded outputs
unit_type, unit_type_string = bc.qm.get_quality_unit_type(param, quality_metrics)

print('n units:', len(unit_type))
print('template_waveforms shape:', getattr(template_waveforms, 'shape', None))


## 4) Rerun summary plots for any probe after any Bombcell run


In [ ]:
# Correct API call name: plot_summary_data
bc.plot_summary_data(quality_metrics, template_waveforms, unit_type, unit_type_string, param)


## 5) Reload manual-vs-bombcell comparison


In [ ]:
bc.compare_manual_vs_bombcell(str(save_path))


## 6) Launch main/master Bombcell GUI for this run


In [ ]:
gui = bc.unit_quality_gui(
    ks_dir=str(ks_dir),
    quality_metrics=quality_metrics,
    unit_types=unit_type,
    param=param,
    save_path=str(save_path),
)
gui


## 7) Label units by distance from probe tip (ROI labeling)

Example: Probe B NP1.0 ROI is tip→950um. Units with `distance_from_tip_um <= 950` are labeled `IN_ROI`; others are `OUTSIDE_ROI`.


In [ ]:
import sys
from pathlib import Path
import pandas as pd

analysis_dir = (Path.cwd() / '..' / 'analyzing_BC_results').resolve()
if not analysis_dir.exists():
    raise FileNotFoundError(f'Expected analyzing_BC_results folder at: {analysis_dir}')
sys.path.insert(0, str(analysis_dir))
from post_analysis_setup import label_units_by_tip_distance

# ROI default comes from JSON: cfg['probe_recording_roi'][TARGET_PROBE]
roi_defaults = cfg.get('probe_recording_roi', {})
ROI_END_UM_DEFAULT = roi_defaults.get(TARGET_PROBE, None)
ROI_END_UM_OVERRIDE = None  # set a number here to override JSON for this notebook run
ROI_END_UM = ROI_END_UM_DEFAULT if ROI_END_UM_OVERRIDE is None else ROI_END_UM_OVERRIDE

if ROI_END_UM is None:
    raise ValueError(f'No ROI configured for probe {TARGET_PROBE}. Add probe_recording_roi[{TARGET_PROBE}] in JSON or set ROI_END_UM_OVERRIDE.')

qm_with_roi = label_units_by_tip_distance(
    quality_metrics=quality_metrics,
    ks_dir=ks_dir,
    roi_end_um=ROI_END_UM,
    tip_position='min_y',
    in_label='IN_ROI',
    out_label='OUTSIDE_ROI',
)

print(f'Using ROI_END_UM={ROI_END_UM} for probe {TARGET_PROBE}')
print(qm_with_roi['roi_label'].value_counts(dropna=False))
display(qm_with_roi[['maxChannels', 'distance_from_tip_um', 'roi_label']].head(10))

# Optional: save alongside your probe exports
roi_csv = Path(save_path) / f'Probe_{TARGET_PROBE}_quality_metrics_with_roi.csv'
qm_with_roi.to_csv(roi_csv, index=False)
print('Saved:', roi_csv)
